In [1]:
import pandas as pd
import numpy as np
import random

import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.51-b03, mixed mode)
  Starting server from C:\Users\Acer\anaconda3\envs\engine\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Acer\AppData\Local\Temp\tmpwcw2rhgo
  JVM stdout: C:\Users\Acer\AppData\Local\Temp\tmpwcw2rhgo\h2o_Acer_started_from_python.out
  JVM stderr: C:\Users\Acer\AppData\Local\Temp\tmpwcw2rhgo\h2o_Acer_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Bangkok
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 3 days
H2O_cluster_name:,H2O_from_python_Acer_lm2ykq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.501 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [2]:
tasks = pd.read_csv('data/30_2018/task_data.csv')
tasks_df = pd.read_csv('data/30_2018/task_train.csv')

target = 'TaskDelay'

X = tasks_df.copy()
y = X.pop(target)

In [3]:
project_ids = tasks['ProjectID'].unique().tolist()
indices = random.sample(project_ids, int(0.2 * len(project_ids)))
val_indices = random.sample(indices, int(0.5 * len(indices)))
test_indices = [x for x in indices if x not in val_indices]
val_idx = tasks.loc[tasks['ProjectID'].isin(val_indices)].index.tolist()
test_idx = tasks.loc[tasks['ProjectID'].isin(test_indices)].index.tolist()

val_frame = tasks_df[tasks_df.index.isin(val_idx)]
test_frame = tasks_df[tasks_df.index.isin(test_idx)]
train_frame = tasks_df[(~tasks_df.index.isin(test_idx)) & (~tasks_df.index.isin(val_idx))]

In [4]:
train_h2o = h2o.H2OFrame(train_frame)
val_h2o = h2o.H2OFrame(val_frame)
test_h2o = h2o.H2OFrame(test_frame)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
x = train_h2o.columns
y = 'TaskDelay'  # replace with your actual response column
x.remove(y)

In [6]:
aml = H2OAutoML(max_runtime_secs=1800,
                seed=42,
                stopping_metric='deviance', 
                nfolds=0)
aml.train(x=x, y=y, training_frame=train_h2o, validation_frame=val_h2o)

AutoML progress: |
18:49:44.584: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_1_20240516_184944_model_9


Status of Neuron Layers: predicting TaskDelay, regression, gaussian distribution, Quadratic loss, 1,301 weights/biases, 21.2 KB, 8,800,007 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate               rate_rms               momentum    mean_weight             weight_rms           mean_bias             bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ----------------------  ---------------------  ----------  ----------------------  -------------------  --------------------  -----------------------
    1        11       Input             0.0
    2        100      RectifierDropout  50.0       0.0   0.0   0.0007236248632067476   0.00086931511759758    0.0         -0.0044041509155993645  0.23615771532058716  0.051110675882042766  0.14817923307418823
    3        1        Linear                       0.0   0.0   0.00011331973790220218  6.479423609562218e-05  0.0         0.007632394535467029    0.11308878660202026  -0.01096037742711828  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 1.563019467377836
RMSE: 1.2502077696838378
MAE: 0.9174833059769583
RMSLE: 0.44637888330864695
Mean Residual Deviance: 1.563019467377836

ModelMetricsRegression: deeplearning
** Reported on validation data. **

MSE: 2.12595370430176
RMSE: 1.4580650548935599
MAE: 1.1196588044441373
RMSLE: 0.5248728101232691
Mean Residual Deviance: 2.12595370430176

Scoring History: 
    timestamp            duration          training_speed    epochs    iterations    samples      training_rmse    training_deviance    training_mae    training_r2    validation_rmse    validation_deviance    validation_mae    validation_r2
--  -------------------  ----------------  ----------------  --------  ------------  -----------  ---------------  -------------------  --------------  -------------  -----------------  ---------------------  ----------------  ---------------
    2024-05-16 18:58:48  0.000 sec                           0         0             0            nan              nan                  nan             nan            nan                nan                    nan               nan
    2024-05-16 18:58:49  5 min 40.131 sec  175943 obs/sec    4.79378   1             99936        1.35354          1.83208              1.05342         0.435617       1.55873            2.42963                1.26157           0.343168
    2024-05-16 18:58:54  5 min 45.479 sec  220927 obs/sec    62.335    13            1.2995e+06   1.25021          1.56302              0.917483        0.518503       1.45807            2.12595                1.11966           0.425264
    2024-05-16 18:58:59  5 min 50.680 sec  244204 obs/sec    129.5     27            2.69968e+06  1.22337          1.49664              0.905749        0.538952       1.47387            2.17229                1.1463            0.412737
    2024-05-16 18:59:04  5 min 55.896 sec  258586 obs/sec    201.466   42            4.19996e+06  1.22979          1.5124               0.893419        0.534099       1.47524            2.17632                1.12868           0.411647
    2024-05-16 18:59:09  6 min  0.983 sec  267605 obs/sec    273.394   57            5.69945e+06  1.21702          1.48113              0.898011        0.543729       1.4688             2.15738                1.13621           0.416767
    2024-05-16 18:59:15  6 min  6.303 sec  270813 obs/sec    345.353   72            7.19958e+06  1.20974          1.46346              0.8907          0.549172       1.46412            2.14364                1.13394           0.420481
    2024-05-16 18:59:20  6 min 11.480 sec  277357 obs/sec    422.123   88            8.80001e+06  1.21501          1.47626              0.886876        0.545231       1.49086            2.22267                1.14845           0.399117
    2

In [7]:
lb = aml.leaderboard
print(lb)

model_id                                                  rmse      mse      mae     rmsle    mean_residual_deviance
DeepLearning_grid_1_AutoML_1_20240516_184944_model_9   1.45807  2.12595  1.11966  0.524873                   2.12595
DeepLearning_grid_1_AutoML_1_20240516_184944_model_28  1.46262  2.13924  1.13157  0.534913                   2.13924
GBM_grid_1_AutoML_1_20240516_184944_model_154          1.47294  2.16955  1.13645  0.53162                    2.16955
DeepLearning_grid_3_AutoML_1_20240516_184944_model_2   1.47312  2.17009  1.12319  0.524545                   2.17009
GBM_grid_1_AutoML_1_20240516_184944_model_177          1.47395  2.17251  1.15012  0.53636                    2.17251
GBM_grid_1_AutoML_1_20240516_184944_model_191          1.4752   2.17621  1.14921  0.538846                   2.17621
GBM_grid_1_AutoML_1_20240516_184944_model_182          1.47527  2.17642  1.1475   0.5378                     2.17642
DeepLearning_grid_1_AutoML_1_20240516_184944_model_6   1.47578  

In [8]:
best = aml.leader
best.model_performance(test_h2o)

ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 1.7424421850688505
RMSE: 1.32001597909603
MAE: 0.9964841344157069
RMSLE: 0.465604481532183
Mean Residual Deviance: 1.7424421850688505

In [9]:
h2o.save_model(model=best, path='model', force=True)

'D:\\Work\\Angusta System\\Code\\model\\DeepLearning_grid_1_AutoML_1_20240516_184944_model_9'

In [10]:
loaded_model = h2o.load_model("model/30_2018_30")

In [11]:
loaded_model.model_performance(test_h2o)

ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 1.7424421850688505
RMSE: 1.32001597909603
MAE: 0.9964841344157069
RMSLE: 0.465604481532183
Mean Residual Deviance: 1.7424421850688505

In [12]:
preds = loaded_model.predict(test_h2o)
test_h2o['Preds'] = preds 
test_h2o.head(50)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


Cost,Priority,Progress,Duration,Worker,TaskLength,IsBadWeather,WeatherAssessment,StartDelay,DayCount,TaskDelay,Is_Delayed,Preds
2271,1,11.1111,9,5,0,1,37.5,0,0,1,0,1.34924
2271,1,22.2222,9,5,0,1,37.5,0,1,1,0,1.32734
2271,1,33.3333,9,5,0,1,37.5,0,2,1,0,1.3066
2271,1,33.3333,9,5,0,0,37.5,0,3,1,0,1.60341
2271,1,44.4444,9,5,0,0,37.5,0,4,1,0,1.52661
2271,1,55.5556,9,5,0,0,37.5,0,5,1,0,1.44549
2138,1,0,8,5,0,0,25,0,0,3,0,1.73432
2271,1,66.6667,9,5,0,0,37.5,0,6,1,0,1.33433
2138,1,12.5,8,5,0,0,25,0,1,3,0,1.65777
2271,1,77.7778,9,5,0,0,37.5,0,7,1,0,1.14891
